In [1]:
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import StandardScaler

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()


url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)
dataset = raw_dataset.copy()
dataset = dataset.dropna()

dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
dataset = pd.get_dummies(dataset, columns=['Origin'], prefix='', prefix_sep='')
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')

normalizer = StandardScaler()
normalizer.fit(train_features)
train_features = normalizer.transform(train_features)
test_features = normalizer.transform(test_features)

In [2]:
train_labels

In [3]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_features, train_labels)
print(f"R2 = {lr.score(test_features, test_labels)}")

from sklearn.metrics import mean_squared_error

print(f"MSE = {mean_squared_error(lr.predict(test_features), test_labels)}")

In [4]:
from keras import layers

linear_model = keras.Sequential([
    layers.Dense(units=16, activation='relu'),
    layers.Dropout(rate=0.2),
    # layers.Dense(units=16, activation='relu'),
    # layers.Dropout(rate=0.2),
    layers.Dense(units=1),
])

# linear_model = keras.Sequential([
#     normalizer,
#     layers.Dense(units=1),
# ])

linear_model.compile(    
    optimizer=keras.optimizers.legacy.Adam(learning_rate=0.1),
    loss='mse')

In [5]:
history = linear_model.fit(
    train_features,
    np.array(train_labels),
    epochs=100,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [6]:
pd.DataFrame(history.history).plot()
pd.DataFrame(history.history).tail()

In [7]:
train_features

In [8]:
from disentanglement.models.architectures import get_dropout_from_backbone
from disentanglement.models.gaussian_logits_models import train_gaussian_logits_model
from tensorflow.keras import backend as K


# disable_eager_execution()
def get_auto_mpg_backbone(n_training_samples=0):
    linear_model = keras.Sequential([
        layers.Input(9),
        layers.Dense(units=16, activation='relu')])
    return linear_model

In [9]:
def architecture(**_):
    return get_dropout_from_backbone(backbone_func=get_auto_mpg_backbone, hidden_size=16)

In [10]:
model = train_gaussian_logits_model(architecture, np.array(train_features), np.array(train_labels), n_classes=0, epochs=100_000, regression=True)

In [11]:
preds, ale_std, epi_std = model.predict(np.array(test_features), num_samples=100, disentangle_uncertainty=True)

In [12]:
mean_squared_error(preds, test_labels,)

In [13]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(preds, test_labels)

In [14]:
print(ale_std.mean())
print(epi_std.mean())

In [15]:
np.array(train_features).shape